In [6]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import *
import time
import utilities as utl
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph


# from fast_generator import fc_tree_commute_recur_lookahead_fast

# from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
# from circuit_generator import construct_qcc_circuit

import numpy as np
# from rustiq import pauli_network_synthesis, Metric
# from rustiq.utils import entangling_count, entangling_depth

import os
import importlib
from copy import deepcopy
# import sys
# from __future__ import absolute_import
current_dir=os.getcwd()
os.chdir("../vqe_tetris/vqe_tetris-master/core")
from utils import synthesis_lookahead, hardware, synthesis_max_cancel
from benchmark.mypauli import pauliString
importlib.reload(synthesis_lookahead)
# from tools import print_qc
os.chdir(current_dir)

In [7]:
# Tetris
# ctime = time.time
def Tetris_lookahead_Mahattan(parr, num_qubits, use_bridge, swap_coefficient=3, k=10):
    # print('Tetris passes, Our schedule, Our synthesis, mahattan', flush=True)
    # lnq = len(parr[0][0])
    # length = lnq // 2 # `length' is a hyperparameter, and can be adjusted for best performance. Here we keep `length' fixed for simplicity.
    # coup = load_coupling_map('manhattan')
    # t0 = ctime()
    # a2 = gate_count_oriented_scheduling(parr)#, length=length, maxiter=30)
    # a2 = [block for blocks in a2 for block in blocks]
    # print("length", len(parr[0]))
    pauli_map=list(range(num_qubits))
    graph_full_connected=build_graph(num_qubits)
    if k>len(parr):
        k=len(parr)-1
    qc, metrics = synthesis_lookahead.synthesis_lookahead(parr, graph=graph_full_connected, arch='manhattan', 
                    use_bridge=False, pauli_map=pauli_map, swap_coefficient=swap_coefficient, k=k)
    # qc, metrics = synthesis_max_cancel.synthesis_max_cancel(parr, graph=graph_full_connected, arch='manhattan', 
    #                 use_bridge=False, pauli_map=pauli_map)

    return qc


def build_graph(n):
    '''G adjacency matrix and C cost matrix. For a fully connected device.'''
    G=np.ones((n,n))-np.eye(n)
    C=np.zeros((n,n)) #no cost
    return hardware.pGraph(G, C)

def is_all_identity(pauli):
    return all(char == 'I' for char in pauli)


# def update_json_file(filename, runtime, cx_count, circ_depth):
#     with open(f'benchmarks/results/test_' + filename, 'r') as paulis_file:
#         data=json.load(paulis_file)
#     print(data[1])
#     # with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
#     #     json.dump([test_paulis, results], paulis_file, indent=4)

In [14]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                test_paulis = json.load(file)
            # Filter the list to remove all identity Paulis
            num_qubits=len(test_paulis[0])
            paulis = [[pauliString(p)] for p in test_paulis if not is_all_identity(p)]
            number_of_ham=len(paulis)

            # Measure time for Tetris method
            start_time = time.time()
            circuit_tetris = Tetris_lookahead_Mahattan(paulis, num_qubits, False)
            circuit_tetris = transpile(circuit_tetris, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
            end_time = time.time()
            tetris_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": number_of_ham,
                "times": {
                    # "our_time": our_time,
                    # "combined_time": combined_time,
                    # "qiskit_time": qiskit_time,
                    # "rustiq_time": rustiq_time
                    "tetris_time": tetris_time
                },
                "gate_counts": {
                    # "our_method": opt_qc_f.count_ops().get('cx', 0),
                    # "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    # "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    # "rustiq_method": entangling_count(circuit),
                    "tetris_method": circuit_tetris.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    # "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    # "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    # "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    # "rustiq_method": entangling_depth(circuit),
                    "tetris_method": circuit_tetris.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results/test_tetris_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_tetris_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [9]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, filename = "Paulis"):

    results = []
    # paulis = test_paulis
    # Filter the list to remove all identity Paulis
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]
    
    number_of_ham=len(test_paulis)
    # Measure time for Tetris method
    num_qubits=len(test_paulis[0])
    paulis = [[pauliString(p)] for p in test_paulis]
    start_time = time.time()
    circuit_tetris = Tetris_lookahead_Mahattan(paulis, num_qubits, False)
    circuit_tetris = transpile(circuit_tetris, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
    end_time = time.time()
    tetris_time = end_time - start_time

    # Collect results
    result = {
        "num_paulis": number_of_ham,
        "times": {
            # "our_time": our_time,
            # "combined_time": combined_time,
            # "qiskit_time": qiskit_time,
            # "rustiq_time": rustiq_time
            "tetris_time": tetris_time
        },
        "gate_counts": {
            # "our_method": opt_qc_f.count_ops().get('cx', 0),
            # "combined_method": opt_qiskit.count_ops().get('cx', 0),
            # "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            # "rustiq_method": entangling_count(circuit),
            "tetris_method": circuit_tetris.count_ops().get('cx', 0)
        },
        "circuit_entangling_depth": {
            # "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            # "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            # "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            # "rustiq_method": entangling_depth(circuit),
            "tetris_method": circuit_tetris.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results/test_tetris_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_tetris_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return circuit_tetris #sorted_entanglers_f
    

    

In [15]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20]#, 24]
# paulis_len=[24, 80, 320, 1656, 5376, 13400]
#First evaluate the UCCSD ansatz:
for e, o in zip(electrons_list, orbitals_list):
    filename=f"uccsd_hamiltonian_e{e}_o{o}.json"
    with open(f"benchmarks/uccsd_hamiltonians/{filename}", "r") as file:
        data=json.load(file)
    # test_paulis = generate_UCCSD_entanglers(x, y)
    print(data)
    data = [p for block in data for p in block if not is_all_identity(p)]
    num_hams=len(data)
    print(data)
    entanglers = run_experiment_paulis(data, save_output = True, filename=f"Paulis{num_hams}.json")

[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
['YZXI', 'XZXI', 'YZYI', 'XZYI', 'IYZX', 'IXZX', 'IYZY', 'IXZY', 'YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']
24
{'num_paulis': 24, 'times': {'tetris_time': 0.8508553504943848}, 'gate_counts': {'tetris_method': 53}, 'circuit_entangling_depth': {'tetris_method': 53}, 'test_paulis_file': 'benchmarks/results/test_tetris_Paulis24.json'}
[['YZXIII', 'XZXIII', 'YZYIII', 'XZYIII'], ['YZZZXI', 'XZZZXI', 'YZZZYI', 'XZZZYI'], ['IYZXII', 'IXZXII', 'IYZYII', 'IXZYII'], ['IYZZZX', 'IXZZZX', 'IYZZZY', 'IXZZZY'], ['YXYXII', 'XXYXII', 'YYYXII', 'XYYXII', 'YXYYII', 'XXYYII', 'YYYYII', 'XYYYII', 'YXXXII', 'XXXXII', 'YYXXII', 'XYXXII', 'YXXYII', 'XXXYII', 'YYXYII', 'XYXYII'], ['YXYZZX', 'XXYZZX', 'YYYZZX', 'XYYZZX', 'YXYZZY', '

KeyboardInterrupt: 

In [16]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True)

benzene.json
1254
{'num_paulis': 1254, 'times': {'tetris_time': 57.38052010536194}, 'gate_counts': {'tetris_method': 7532}, 'circuit_entangling_depth': {'tetris_method': 7526}, 'test_paulis_file': 'benchmarks/results/test_tetris_benzene.json'}
H2O.json
184
{'num_paulis': 184, 'times': {'tetris_time': 6.693113088607788}, 'gate_counts': {'tetris_method': 872}, 'circuit_entangling_depth': {'tetris_method': 870}, 'test_paulis_file': 'benchmarks/results/test_tetris_H2O.json'}
LiH.json
61
{'num_paulis': 61, 'times': {'tetris_time': 1.6615588665008545}, 'gate_counts': {'tetris_method': 215}, 'circuit_entangling_depth': {'tetris_method': 214}, 'test_paulis_file': 'benchmarks/results/test_tetris_LiH.json'}


In [ ]:
# #Then compare the MAXCUT paulis in max_cut_paulis folder

# run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = False)

In [ ]:
# #Labs

# run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = False)